In [ ]:
# # # !pip install sqlalchemy
# # # !pip install psycopg2-binary
# # # !pip install Flask-SQLAlchemy
# # # !pip3 install sqlalchemy
# # # !pip3 install psycopg2-binary
# # # !pip3 install Flask-SQLAlchemy

# !conda install -y sqlalchemy
# !conda install -y psycopg2-binary

# !conda install -y sqlalchemy
# !conda install -y psycopg2-binary
# !conda install -y Flask-SQLAlchemy

# !pip install python-dotenv

import os

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
host = 'localhost'
port = 5432
database = 'northwind'
user = os.getenv('POSTGRES_USER')
password = os.getenv('POSTGRES_PASSWORD') # set in .env file
password = 'changeme'
uri = f'postgresql://{user}:{password}@{host}:{port}/{database}'

In [ ]:
from sqlalchemy import create_engine

engine = create_engine(uri, echo=False)

In [ ]:
from sqlalchemy import Table
from sqlalchemy import inspect
from sqlalchemy import MetaData

def drop_table_if_exists(table_name):
    if inspect(engine).has_table(table_name):
        table = Table(table_name, MetaData(engine), autoload_with=engine)
        table.drop()

In [ ]:
# reload tables from csv files
data_dir= './var/data/'
files = os.listdir(data_dir)
for file in files:
    full_path = data_dir + file
    table_name = file.split('.')[0]
    ext = file.split('.')[1]
    if ext == 'csv':
        print('from %s file -> to "%s" table' % (full_path, table_name))
        drop_table_if_exists(table_name)
        df = pd.read_csv(full_path, index_col=0)
        df.to_sql(table_name, engine)

In [ ]:
import pandas as pd

query = 'SELECT c."categoryID", c."categoryName", c."description" FROM categories AS c;'

df = pd.read_sql(query, engine)
df.head()